## subclasses, staticmethod, classmethod
Methods without `self`

![MessageBox and ConfirmBox examples](messagebox.svg)

In [1]:
import json

class MessageBox:
    """
    Dialog box with a single button
    """
    def __init__(self, title_text, body_text, dismiss_button="Okay"):
        self.title = title_text
        self.body = body_text
        self.buttons = [dismiss_button]

    def __repr__(self):
        return f'<{self.__class__.__name__} {self.title}>'
    
    @classmethod
    def from_json(cls, jtext):
        if not cls.valid_json(jtext):
            raise ValueError('Invalid JSON for this class')
        return cls(**json.loads(jtext))
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button'}
        return required <= set(data) <= everything

m = MessageBox.from_json('''
{
  "title_text": "Alert",
  "body_text": "Something bad happened"
}''')
m

<MessageBox Alert>

In [2]:
m.buttons

['Okay']

In [3]:
MessageBox.valid_json('42')

False

In [4]:
MessageBox.valid_json('{"title_text":"a","body_text":"b"}')

True

In [5]:
MessageBox.from_json('{"title_text":"hello"}')

ValueError: Invalid JSON for this class

In [6]:
MessageBox.from_json('hello')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
issubclass(json.JSONDecodeError, ValueError)

True

### 🏠🏠🏠 Idiomatic Python: Edge Cases, Duck Typing

- If the exception raised is the right type, consider not special casing every possible error case just to customize your exception messages
- Instead of `isinstance` use the values passed and catch exceptions

In [2]:
class ConfirmBox(MessageBox):
    """
    Dialog box with Proceed/Cancel buttons
    """
    def __init__(self, title_text, body_text,
                 dismiss_button='Cancel', confirm_button='Proceed'):
        super().__init__(title_text, body_text, dismiss_button)
        self.buttons.append(confirm_button)
    
    @staticmethod
    def valid_json(jtext):
        data = json.loads(jtext)
        if not isinstance(data, dict):
            return False
        
        required = {'title_text', 'body_text'}
        everything = required | {'dismiss_button', 'confirm_button'}
        return required <= set(data) <= everything

c = ConfirmBox.from_json('''
{
  "title_text": "Danger",
  "body_text": "Vent radioactive gas?",
  "confirm_button": "Yes I'm sure"
}''')
c

<ConfirmBox Danger>

In [73]:
c.buttons

['Cancel', "Yes I'm sure"]

In [75]:
ConfirmBox.valid_json('''
{
  "title_text": "hi",
  "body_text": "how are you?",
  "confirm_button": "good",
  "dismiss_button": "bad"
}''')

True

### Review
- how do you specify a superclass?
- what is passed as the first parameter to `@classmethod` methods?
- what is passed as the first parameter to `@staticmethod` methods?

## Multiple Inheritance and MRO

![ProjectSettings(AuditDict(dict), ConfigDict(dict))](projectsettings.svg)

In [10]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

class AuditDict(dict):
    """
    Log changes to this dict
    """
    def __setitem__(self, key, value):
        logging.info(f'setting [{key}] to {value}')
        super().__setitem__(key, value)

class ConfigDict(dict):
    """
    Force dict keys to lowercase
    """
    def __setitem__(self, key, value):
        super().__setitem__(key.lower(), value)

a = AuditDict()
a['Test'] = 'hi'

INFO:root:setting [Test] to hi


In [11]:
b = ConfigDict()
b['TEST'] = 'hello'
b

{'test': 'hello'}

In [8]:
class ProjectSettings(AuditDict, ConfigDict):
    pass

c = ProjectSettings()
c['SSL'] = 'enabled'
c

INFO:root:setting [SSL] to enabled


{'ssl': 'enabled'}

In [9]:
class ProjectSettings(ConfigDict, AuditDict):
    pass

d = ProjectSettings()
d['SSL'] = 'enabled'
d

INFO:root:setting [ssl] to enabled


{'ssl': 'enabled'}

In [12]:
ProjectSettings.__mro__

(__main__.ProjectSettings,
 __main__.ConfigDict,
 __main__.AuditDict,
 dict,
 object)

### Review
- what determines the method resolution order for classes?
- what keyword is used for "no operation" blocks?
- what builtin is used to access the next class in the MRO?

### Exercise: course registration 2
1. Add methods to the `Course` class from the previous section to allow accessing students by their id and to get their size:
```python
>>> s = mycourse[97865]
>>> s.name
'Name Here'
>>> len(mycourse)
2
```
2. Add a method to serialize a course as JSON
```python
>>> mycourse.to_json()
'{"students":[12345, 97865]}'
```

## Extra Material

### Custom Containers

```
        BitList
        ┌───┬───┬───┬───┬───┬───┬───┬───┬───┐
0x8F =    1   1   1   1   0   0   0   1   0   ⋯
        └───┴───┴───┴───┴───┴───┴───┴───┴───┘
        0   1   2   3   4   5   6   7   8   9  ⋯

```

In [5]:
class BitList:
    """Mutable sequence of 0's and 1's"""
    value = 0

    def __init__(self, val):
        self.value = int(val)
        
    def __getitem__(self, bit):
        return (self.value >> bit) & 1
    
    def __setitem__(self, bit, val):
        if val == 0:
            self.value &= ~(1 << bit)
        elif val == 1:
            self.value |= (1 << bit)
        else:
            raise ValueError('can only store 0 and 1')
    
bb = BitList(0x80ff)
bb[0]

1

In [6]:
bb[1000]

0

In [8]:
# WARNING would run forever:  list(bb)
# because __getitem__ never raises IndexError

In [13]:
class BitList:
    """Mutable sequence of 0's and 1's"""
    value = 0

    def __init__(self, val):
        self.value = int(val)
        
    def __getitem__(self, bit):
        return (self.value >> bit) & 1
    
    def __setitem__(self, bit, val):
        if val == 0:
            self.value &= ~(1 << bit)
        elif val == 1:
            self.value |= (1 << bit)
        else:
            raise ValueError('can only store 0 and 1')

    def __len__(self):
        return self.value.bit_length()

    def __iter__(self):
        v = self.value
        while v:
            yield v & 1
            v >>= 1

bb = BitList(0x80ff)

In [14]:
list(bb)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]

In [9]:
list(reversed(bb))

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
bb[10] = 1
bb[1] = 0
list(bb)

[1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]

In [17]:
bb['key']

TypeError: unsupported operand type(s) for >>: 'int' and 'str'

### Review
- is `myobject['name']` the same as `myobject.name`?
- what special method name is required for `iter(myobject)`?
- what special method name is required for `myobject[index] = value`?
- what special method name is required for `len(myobject)`?

In [ ]:
# convert BitList to/from bytes
class BitList:
    """Mutable container of 0's and 1's"""
    value = 0

    def __init__(self, val):
        self.value = int(val)
        
    def __getitem__(self, bit):
        return (self.value >> bit) & 1
    
    def __setitem__(self, bit, val):
        if val & ~1:
            raise ValueError('can only store 0 and 1')
        self.value = self.value & ~(1 << bit) | (val << bit)

    def __len__(self):
        return self.value.bit_length()

    def __iter__(self):
        v = self.value
        while v:
            yield v & 1
            v >>= 1

    @classmethod
    def from_bytes(cls, byt):
        obj = cls(int.from_bytes(byt, 'little'))
        return obj

    def __bytes__(self):
        return self.value.to_bytes(
            (self.value.bit_length() + 7) // 8, 'little')

bb = BitList(0x80ff)

bytes(bb)

In [ ]:
list(BitList.from_bytes(b'\xfd\x84'))

### Descriptors
The power behind `property`, `classmethod`, `staticmethod` and normal functions becoming methods

In [ ]:
# descriptors are class attributes with __get__ method
class Tripwire:
    def __get__(self, obj, typ=None):
        print('caught access within', typ)
        if obj:
            print('from object', obj)
        return 'looks legit'

class Building:
    doorway = Tripwire()

b = Building()
# descriptors capture object access
b.doorway

In [ ]:
# descriptors capture class access as well
Building.doorway

In [ ]:
# functions implement __get__ to automatically capture self parameter
def i_could_be_a_method(self):
    print('hello', self)

i_could_be_a_method.__get__

In [ ]:
# __get__ can be called directly to test behaviour
m = i_could_be_a_method.__get__('fake')
m

In [ ]:
# we see "self" was bound to our string
m()

In [ ]:
# descriptors can use the object to store values they need
# tracked per object. Implementing __set__ makes this a data
# descriptor
from itertools import count, repeat

class TicketDispenser:
    def __init__(self, name):
        self.name = name

    def __get__(self, obj, typ=None):
        if not obj:
            raise AttributeError('dispenser requires object')
        # Using __dict__ to store a value with the same name
        # hides these stored values from casual observation
        c = obj.__dict__.setdefault(self.name, count(1))
        return next(c)

    def __set__(self, obj, value):
        obj.__dict__[self.name] = count(value)

    def __delete__(self, obj):
        obj.__dict__[self.name] = repeat('out of order')
        
class ServiceCounter:
    support = TicketDispenser('support')

sc1 = ServiceCounter()
sc2 = ServiceCounter()

print(sc1.support)
print(sc1.support)

In [ ]:
print(sc2.support)
sc2.support = 500
print(sc2.support)
print(sc2.support)
print(sc1.support)
print(sc1.support)

In [ ]:
del sc1.support
print(sc1.support)
print(sc1.support)

In [ ]:
ServiceCounter.support

See also https://docs.python.org/3/howto/descriptor.html#descriptor-protocol

In [ ]:
# slicing uses __getitem__ too
bb[:5]

In [ ]:
# monkey-patch an updated __getitem__
def bitlist_getitem(self, bit_or_slice):
    if isinstance(bit_or_slice, slice):
        return [
            (self.value >> n) & 1 for n in
            range(*bit_or_slice.indices(len(self)))
        ]
    return (self.value >> bit_or_slice) & 1

BitList.__getitem__ = bitlist_getitem

bb[:7]

In [ ]:
# Ellipsis easter egg and explicit type check
def bitlist_getitem(self, index):
    if isinstance(index, slice):
        return [
            (self.value >> n) & 1 for n in
            range(*index.indices(len(self)))
        ]
    
    elif index == ...:
        return ''.join('🐔' if v else '🥚' for v in self)
    
    try:
        return (self.value >> index) & 1
    except TypeError as e:
        raise TypeError('BitList indices must be integers') from e

BitList.__getitem__ = bitlist_getitem

bb[...]

### 🍎🍎🍎 Python Core: ... @

- Ellipsis and the matrix multiplication infix operator aren't used in the standard library, were created for numpy

In [ ]:
# range(slice.indices(...)) takes care of -ve indexes, step size etc.
bb[-5:]

In [ ]:
bb[:]

### Metaclasses
Invent your own mini-language

In [26]:
type(42)

int

In [27]:
type('Thing', (), {})

__main__.Thing

In [34]:
def __init__(self):
    self.x = 1
    self.y = 2
    
Thing = type('Thing', (), {'__init__': __init__})

t = Thing()
print(t.x, t.y)

1 2


In [53]:
class Anything:
    pass

print(type(Anything))
print(type(Thing))

<class 'type'>
<class 'type'>


In [36]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

print(type(MySubclass))

<class '__main__.Meta'>


In [51]:
class Meta(type):
    def __new__(cls, name, bases, namespace):
        print(f'{name!r}\n{bases!r}\n{namespace!r}\n')
        return type.__new__(cls, name, bases, namespace)

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    def greeting(self):
        print('hi')

'MyClass'
()
{'__module__': '__main__', '__qualname__': 'MyClass'}

'MySubclass'
(<class '__main__.MyClass'>,)
{'__module__': '__main__', '__qualname__': 'MySubclass', 'greeting': <function MySubclass.greeting at 0x7f7f066f29d8>}



In [5]:
class FormMeta(type):
    def __new__(cls, name, bases, namespace):
        if not bases:
            return type.__new__(cls, name, bases, namespace)
        
        namespace['_fields'] = {}
        for f in list(namespace):
            if not f.startswith('_'):
                namespace['_fields'][f] = namespace.pop(f)
                
        return type.__new__(cls, name, bases, namespace)

class Form(metaclass=FormMeta):
    def run(self):
        for name, prompt in self._fields.items():
            setattr(self, name, input(prompt + ': '))

class SignupForm(Form):
    name = 'Your full name'
    email = 'Your email address'
    phone = 'Your phone number with area code'

SignupForm._fields

{'name': 'Your full name',
 'email': 'Your email address',
 'phone': 'Your phone number with area code'}

In [6]:
f = SignupForm()
f.name

AttributeError: 'SignupForm' object has no attribute 'name'

In [7]:
f.run()

Your full name: A Person
Your email address: person@example.com
Your phone number with area code: 555-555-5555


In [8]:
f.name

'A Person'

In [9]:
f.email

'person@example.com'

In [11]:
class FunctionForm(metaclass=FormMeta):
    def run(self):
        for name, fn in self._fields.items():
            setattr(self, name, fn(input(fn.__doc__ + ': ')))

class SatisfactionSurvey(FunctionForm):
    def recommend(v):
        'Would you recommend us (Y/N)'
        if v == 'Y' or v == 'N':
            return v
        raise ValueError('Y/N only')
    
    def rating(v):
        'How would you rate our service (1-10)'
        return int(v)

ss = SatisfactionSurvey()
ss.run()

Would you recommend us (Y/N): Y
How would you rate our service (1-10): 9


In [12]:
ss.recommend

'Y'

In [13]:
ss.rating

9

Popular metaclasses:
- [Django Forms](https://docs.djangoproject.com/en/2.2/topics/forms/#the-form-class) and [Models](https://docs.djangoproject.com/en/2.2/topics/db/models/#quick-example)
- [Sqlalchemy declarative_base](https://docs.sqlalchemy.org/en/latest/orm/extensions/declarative/basic_use.html)

### MRO is enforced

In [23]:
class ConfigSettings(LoggedLowerDict):
    pass

In [24]:
class ConfigSettings(LoggedLowerDict, LoggedDict):
    pass

In [25]:
class ConfigSettings(LoggedLowerDict, LowerDict, LoggedDict):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases LoggedDict, LowerDict